In [26]:
import pandas as pd
from parse import parse_htmls
from util import read_place_desc, read_htmls_in, write_places_to_tsv, read_places
from service import PlaceService, SearchEngine
from index import preprocess, Index, TfIdfIndex

import requests
from tqdm import tqdm
import os
import time
import util

# Data collection
##  1.1. Get the list of places





In [ ]:
with open('most_popular_places.txt', 'w') as file:
    for i in tqdm(range(300)):
        
        url = 'https://www.atlasobscura.com/places?page='+str(i+1)+'&sort=likes_count'
        list_page = requests.get(url)
        list_soup = BeautifulSoup(list_page.text)
        list_places = [x['href'] for x in list_soup.find_all('a', {'class':"content-card content-card-place"})]
        for place in tqdm(list_places):
            file.write('https://www.atlasobscura.com' + str(place))
            file.write('\n')

## 1.2. Crawl places

In [ ]:
f= open('most_popular_places.txt', 'r')
for j, url in enumerate(f):
    
    if j %18 == 0:
        dir_path = f'page{j//18+1}'
        if not os.path.isdir(dir_path):
            os.mkdir(dir_path)
            
    response = requests.get(url.strip())
    with open(f'{dir_path}/{j+1}.html','w') as file:
        print()
        file.write(response.text)
        
    time.sleep(1)

# 1.3 Parse Pages

In [3]:
#all_htmls = read_htmls_in('./pages')

In [2]:
#%store -r htmls

In [4]:
#all_places = parse_htmls(htmls)

In [5]:
#tsv_path = write_places_to_tsv('./', all_places)

In [263]:
#%store htmls

Stored 'htmls' (list)


In [27]:
ids, descriptions = read_place_desc('./places.tsv')
place_service = PlaceService()
place_service.load('./places.tsv')

# Create Conjunctive Index

In [3]:
index = Index.create_from(ids, descriptions)

# Create Search Engine

In [4]:
search_engine = SearchEngine(index, place_service)

# Run Search

In [5]:
search_engine.query('american museum')

,name,desc,url
6638,Thorne Miniature Rooms,In the depths of the Museum of the Art Institu...,https://www.atlasobscura.com/places/thorne-min...
2920,Grace Hopper&#39;s Bug,"On September 9, 1947, Harvard’s Mark II Aiken ...",https://www.atlasobscura.com/places/grace-hopp...
6306,Biosphere of Montreal,As their contribution to Montreal’s 1967 World...,https://www.atlasobscura.com/places/biosphere-...
5999,The Witch House of Salem,The Salem witchcraft trials took place between...,https://www.atlasobscura.com/places/witch-hous...
4745,Susanna Dickinson Museum,According to a plaque outside her last survivi...,https://www.atlasobscura.com/places/susanna-di...
...,...,...,...
576,Truth or Consequences,Located along the Rio Grande in middle of the ...,https://www.atlasobscura.com/places/truth-or-c...
718,"Basilica of Saint Lawrence, Asheville",The turn of the 20th century was a time of pro...,https://www.atlasobscura.com/places/basilica-o...
359,Stained Glass at Navy Pier,The very first American exhibit dedicated sole...,https://www.atlasobscura.com/places/stained-gl...
516,Abandoned Comet Diner,Just down the street from Mark Twain’s house a...,https://www.atlasobscura.com/places/abandoned-...


# Create TF-IDF Index

In [28]:
tf_idf_index = TfIdfIndex.create_from(ids, descriptions)

# Create TF-IDF Search Engine

In [29]:
tf_idf_search_engine = SearchEngine(tf_idf_index, place_service)

In [30]:
tf_idf_search_engine.query_top_k("american museum", 10)

,name,desc,url,similarity
3926,Smithsonian Sushi Collection,The American History Museum has collected an a...,https://www.atlasobscura.com/places/smithsonia...,0.999944
4697,Zippo/Case Museum,Invented in and still proudly manufactured in ...,https://www.atlasobscura.com/places/zippo-case...,0.998837
2458,Tamástslikt Cultural Institute,"The Tamástslikt Cultural Institute, situated o...",https://www.atlasobscura.com/places/tamastslik...,0.998837
6489,Mercer Museum and Fonthill Castle,"Henry Chapman Mercer, a renowned archaeologist...",https://www.atlasobscura.com/places/fonthill,0.998837
6238,Oak Ridge &quot;The Secret City&quot;,The city of Oak Ridge was established by the U...,https://www.atlasobscura.com/places/the-secret...,0.998837
238,Off the Rez Cafe,The U.S. government’s forced relocation of Nat...,https://www.atlasobscura.com/places/off-the-re...,0.998837
5429,Old Time Wooden Nickel Company,"The adage goes, “don’t take any wooden nickels...",https://www.atlasobscura.com/places/old-time-w...,0.994973
5068,Self-Taught Genius Gallery,"In 2017, the American Folk Art Museum in Manha...",https://www.atlasobscura.com/places/self-taugh...,0.992310
5802,Maryhill Museum and Stonehenge,"“What in Sam Hill?” This odd expression, of wh...",https://www.atlasobscura.com/places/maryhill-m...,0.988467
3701,Museum of Chinese in America,The Museum of Chinese in America is nestled—al...,https://www.atlasobscura.com/places/museum-of-...,0.988467
